# Telecom X — Churn de Clientes (ETL + EDA)

Este notebook implementa o processo de ETL (Extração, Transformação, Carga) e a Análise Exploratória de Dados (EDA) para o problema de evasão de clientes (Churn) da Telecom X.

## 1) Importação de bibliotecas

In [ ]:
import os
import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2) Extração dos Dados (E — Extract)

In [ ]:
# Exemplo: extraindo dados de um CSV hospedado no GitHub
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df_raw = pd.read_csv(url)
df_raw.head()

## 3) Transformação dos Dados (T — Transform)

In [ ]:
# Limpeza e padronização
df = df_raw.copy()

# Corrigir valores vazios em TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Mapear variáveis categóricas Yes/No para 1/0
bool_map = {"Yes": 1, "No": 0}
for col in ['Partner','Dependents','PhoneService','PaperlessBilling','Churn']:
    df[col] = df[col].map(bool_map)

# Feature extra: ticket médio anual
df['AvgAnnualSpend'] = df['MonthlyCharges'] * 12

df.head()

## 4) Carga (L — Load)

In [ ]:
# Salvar dataset tratado
df.to_csv("churn_clean.csv", index=False)
df.to_parquet("churn_clean.parquet", index=False)
print("Dados tratados salvos com sucesso!")

## 5) Análise Exploratória (EDA)

In [ ]:
# Taxa de churn
churn_rate = df['Churn'].mean()
print(f"Taxa de churn: {churn_rate:.2%}")

# Distribuição de tenure
plt.figure(figsize=(6,4))
sns.histplot(df['tenure'], bins=30, kde=False)
plt.title('Distribuição de Tenure (meses)')
plt.show()

In [ ]:
# Boxplot: MonthlyCharges por Churn
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='Churn', y='MonthlyCharges')
plt.title('Monthly Charges por Churn')
plt.show()

In [ ]:
# Churn por tipo de contrato
ct = pd.crosstab(df['Contract'], df['Churn'], normalize='index')
ct.plot(kind='bar', stacked=True, figsize=(6,4))
plt.title('Churn por Tipo de Contrato')
plt.ylabel('Proporção')
plt.show()

## 6) Conclusão
- O churn geral foi calculado.
- Clientes com contrato *Month-to-month* apresentam maior taxa de evasão.
- O tempo de permanência (tenure) também se mostrou relevante: clientes recentes tendem a sair mais.
- Charges mais altas podem estar associadas a maior churn em alguns segmentos.

Esses achados podem direcionar campanhas de retenção, ajustes de contratos e ofertas personalizadas.